#Agentic AI
In this notebook we will make use of the LangChain toolkit (https://python.langchain.com/) to develop various LLM powered applications.

##Installing LangChain and Configuring an LLM

First let's install LangChain:

In [ ]:
!pip install langchain

LangChain needs an LLM to run, and thus we will need to provide it with an endpoint where it can access an LLM.
This endpoint could be either:
- a commercial API, such as the ones of OpenAI, Anthropic, Google, etc.
- or by an opensource LLM (e.g. Llama3.2) that is run locally using the deployment library Ollama.

We will try using a commercial API to start with. Google provides a free tier version of its Gemini API that we can use, so we'll try that one.
- Let's first install the required library from LangChain to access Google's GenAI:

In [ ]:
!pip install langchain-google-genai

To access Google's GenAI API you will need an API key:
- To get a key, go to Google's API Studio (https://aistudio.google.com/) and create a new API key.
- Once you have it, copy the key into the code snippet below.

In [ ]:
API_KEY="..."

We can now create a Gemini2 LLM instance by connecting to the API as follows:

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=API_KEY)

Let's have a quick look at the model:

In [ ]:
llm

Well that doesn't tell us much. It's not an open-source model, so Google won't tell us exactly what architecture they are using anyway ...

If you look at the "gemini-2.0-flash" model here (https://ai.google.dev/gemini-api/docs/models?hl=en#gemini-2.0-flash), we see that the model:  
- can read in a document of up to a million tokens in length (which is a lot!), and produce an ouput of up to 8 thousand tokens;
- can take as input also audio, images and video, as well as text;
- was trained on data up to August 2024.

We can start to chat with the LLM as follows:

In [ ]:
print(llm.invoke("What are the capitals of Australia, Canada, Brasil and South Africa?").content)

Phew! It got those right at least ...

We can see the entire message returned by the model along with the metadata as follows:

In [ ]:
llm.invoke("What are the capitals of Australia, Canada, Brasil and South Africa?")

More importantly, we can see how funny this model is ...

In [ ]:
llm.invoke("Tell me a funny joke about Google, OpenAI and a big bag of bananas.").content

These LLMs are not very funny ...

To make the LLM more useful, we'll need to prompt the model using a proper chat template. In LangChain we can do that by defining a prompts as follows:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Pretend you're a drunken pirate, with a predilection to speak in rhyme and include the term \"whippersnapper\" frequently in conversation."),
    ("user", "{input}")
])

And then define a pipeline, called a "chain" in LangChain, using the pipe operator "|" as follows:

In [ ]:
chain = prompt | llm

Now instead of invoking the LLM directly, we can invoke the chain, which will apply the appropriate prompt that we defined above:

In [ ]:
response = chain.invoke({"input": "Tell me about the time you caught a whale with a fishing rod."}).content
print(response)

Well, that was interesting. What if we ask a more serious question:

In [ ]:
response = chain.invoke({"input": "When was your database last updated?"}).content
print(response)

Looks like we might need to change that system prompt before we try to use this LLM for serious applications ...

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer all questions to the best of your ability."),
    ("user", "{input}")
])
chain = prompt | llm

Let's check that the prompt has been updated:

In [ ]:
response = chain.invoke({"input": "When was your database last updated?"}).content
print(response)

That looks more reasonable ;-)

##Introducing Tools

The reason for using LangChain is to build applications on top of LLMs.
- To make these applications useful, they will need to access additional resources (referred to as *tools* in LangChain) that they can use to solve problems.
- We'll now motivate the need for such tools with a simple example. Consider the following request to the LLM:

In [ ]:
response = chain.invoke({"input": "At an annual interest rate of 95% interest, what would 50 dollars be worth in 10 years time?"}).content
print(response)

OK, that looks good. Let's just do a quick check to make sure that it got the answer right:

In [ ]:
print("$50 after 10 years at 95% interest rate would be: ",50*((1 + 95/100)**10))

Oh no! The LLM had the right calculation, but didn't calculate the right value. The value had the right order of magnitude, but was off by quite a large percentage. This is because the model didn't have access to a calculator to perform the exponentiation calculation.

Let's see if we can help the model by giving it access to some **tools** for computing the compound interest formula.
- A tool in LangChain is simply a method for calculating something.
- Tools are just computations of some form that can be implemented in Python code.
- (See the LangChain documentation here for more information: https://python.langchain.com/docs/how_to/tools_prompting/)

Let's define two tools (functions that the model might make use of) that perform the compound interest calculation and the exponentiation calculation, which is difficult for an LLM to learn how to do automatically:

In [ ]:
from langchain_core.tools import tool

@tool
def compound_interest(principal: float, rate: float, years: float) -> float:
   """
   Computes result of applying particular interest rate to a principal for a number of years.
   First argument is the principal, second is the rate, and the third is the number of years.
   Equation calculated is: output = principal * (1+rate/100)^years.
   """
   return principal * ((1+rate/100) ** years)

@tool
def exponentiate(base: float, power: float) -> float:
   """Computes base^power, i.e. raises the first argument to the power of the second argument."""
   return base ** power

Note the:
- the use of the "**@tool**" keyword (known as a 'decorator') before the definition of the function,
- the specification of the **types** of the input arguments and returned value from the function,
- the addition of a textual description **explaining** what operation is performed by the function.

Once we have defined the tools, we can check that they function as desired by directly invoking them:


In [ ]:
print(compound_interest.invoke({"principal": 50, "rate": 95, "years": 10}))

print(exponentiate.invoke({"base": 1.95, "power": 10}))

Which was the value we saw before, when we calculated the power directly.

Now in order for the LLM to make use of the tools, we need to tell it about them.
- First thing, let's get a list of the available tools and print out their specications:

In [ ]:
tools = [compound_interest, exponentiate]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

LangChain provides a specific format for printing tool information. Let's try it:

In [ ]:
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

Now in order for the LLM to know that the tools are available:
- We will need to let it know about their existance and how to use them in the prompt.
- We can do that as follows:

In [ ]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use.
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding
to the argument names and the values corresponding to the requested values.
"""

Now we need to update the chain with the new prompt:

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

chain = prompt | llm

And let's invoke the chain to see if anything has changed:

In [ ]:
response = chain.invoke({"input": "At an annual interest rate of 95% interest, what would 50 dollars be worth in 10 years time?"}).content
print(response)

Well, that certainly looks different now!
- Instead of returning text, the model has returned a JSON object that calls one of the tools we provided!
- Note that the LLM hasn't actually called the tool. It has just performed the task we told it to do, which is to format a request to the tool.
- In order to make use of this new structured output, we'll need some code to parse the JSON object. We can simply add a JSON parser to the end of our chain:



In [ ]:
from langchain_core.output_parsers import JsonOutputParser

chain_with_parser = prompt | llm | JsonOutputParser()
chain_with_parser.invoke({"input": "At an annual interest rate of 95% interest, what would 50 dollars be worth in 10 years time?"})

Great. We now have the parsed JSON object, but we still need to invoke the tool that is specified in it.
- To do this we will need some code that can can extract the tool name and arguments from the object and call the method associated with a tool.
- This can be done with some Python code as follows:

In [ ]:
from typing import Any, Dict, Optional, TypedDict
from langchain_core.runnables import RunnableConfig

class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""
    name: str
    arguments: Dict[str, Any]

def invoke_tool(tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

OK, so we now have a method called "invoke_tool" that can invoke the tool requested in the JSON object.
- Let's add it to the chain:

In [ ]:
chain_with_parser_and_invoker = prompt | llm | JsonOutputParser() | invoke_tool

In [ ]:
chain_with_parser_and_invoker.invoke({"input": "At an annual interest rate of 95% interest, what would 50 dollars be worth in 10 years time?"})

Few, that worked!

How cool is that!?
- The LLM has decided it needed to use one of the methods we had provided it in order to answer the question,
- and in using it (with a little JSON plumbing help from us), it was able to produce the correct value.

Oftentimes in real applications, we wouldn't stop there, but would want the conversation with the chatbot to continue.
- That would involve adding the output of the tool to the conversation, so that the chatbot could make use of it in responding directly to the user.
- We won't set up the whole conversation for the moment (updates to the LangChain API have made that easy to do recently, so we might return to them at the end of the tutorial).
- For the moment we'll show that it's possible to pass the arguements to the function through the chain along with the output of the function call, so that all the information could be added to a prompt if we wanted it. We can pass the informatoin using a different method from LangChain:

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain2 = prompt | llm | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)

OK, let's try iit with a different request this time. One that ought make use of the other tool we provided:

In [ ]:
chain2.invoke({"input": "What would the number 1.95 raised to the power of 10 be?"})

Note that output contains the arguments to the function as well as the output, so that would could, if we wanted to, use this information directly in a prompt (along with the original question from the user) to generate a new output from the LLM.

Note that this type of interaction is exactly what is needed to make a RAG (Retrieval Augmented Generative) model, by creating a tool that can perform search.

##Controlling a Radio

OK, that was a lot of fun, but in real applications these tools can be used for far more than just performing maths calculations. They can even be used to make changes to the "real world", such as to insert new information into a database.

Let's see if we can set up an application in which we try to control another program: In particular, let's try to control a radio that is streaming content from the web.
- First to set up an internet radio player, we create an HTML page and add an 'audio' element to it.
- Then to enable the radio to be controlled programatically, we can add a *script* that listens for commands on a 'broadcast' channel as follows:  

In [ ]:
from IPython.display import HTML

stream_url = "https://stream3.thesocalsound.org/1"

HTML(f"""

<audio id="player" controls autoplay>
  <source src="{stream_url}" type="audio/mpeg">
  Your browser does not support the audio element.
</audio>

<script>
  var player = document.getElementById('player');
  // control player from other cells by sending broadcast message
  const listenerChannel = new BroadcastChannel('channel');
  listenerChannel.onmessage = (msg) => {{
    if (msg.data === 'play') {{ player.play();}}
    else if (msg.data === 'pause') {{ player.pause(); }}
    else if (msg.data === 'increase volume') {{ player.volume = player.volume + 0.3; }}
    else if (msg.data === 'decrease volume') {{ player.volume = player.volume - 0.3; }}
  }};
</script>

""")

Now we can control the radio remotely (i.e. programatically from another Python cell) by posting appropriate messages on the broadcast channel using Javascript.
- Try uncommenting different messages below to change the command being sent to the radio player:

In [ ]:
from IPython.display import Javascript

message = 'pause'
#message = 'play'
#message = 'increase volume'
#message = 'decrease volume'

display(Javascript(f"""(new BroadcastChannel('channel')).postMessage('{message}');"""))

Now we can use this remote control functionality to define a set of tools:

In [ ]:
@tool
def play_radio_tool():
  "starts the radio"
  display(Javascript(f"""(new BroadcastChannel('channel')).postMessage('play');"""))

@tool
def pause_radio_tool():
  "pauses the radio"
  display(Javascript(f"""(new BroadcastChannel('channel')).postMessage('pause');"""))

@tool
def increase_radio_volume_tool():
  "increases the radio volume"
  display(Javascript(f"""(new BroadcastChannel('channel')).postMessage('increase volume');"""))

@tool
def decrease_radio_volume_tool():
  "decreases the radio volume"
  display(Javascript(f"""(new BroadcastChannel('channel')).postMessage('decrease volume');"""))


Let's add these new tools to the set of tools available to the LLM by updating the system prompt to include their description:

In [ ]:
tools = [exponentiate, compound_interest, play_radio_tool, pause_radio_tool, increase_radio_volume_tool, decrease_radio_volume_tool]
rendered_tools = render_text_description(tools)

system_prompt = f"""\
You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use.
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding
to the argument names and the values corresponding to the requested values.

If the tool takes no arguments, provide an empty dictionary.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

chain_with_parser_and_invoker = prompt | llm | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)

Note that since these new tools don't actually take any arguments, we have added another line to the prompt to tell the LLM what to do if the tool doesn't take any arguments:
- *'If the tool takes no arguments, provide an empty dictionary.'*
- This was necessary to prevent errors being produced by the JSON parser when no dictionary was provided by the LLM.

And now let's see if the LLM is capable of understanding instructions to control the radio.
- Can it turn the radio on?

In [ ]:
chain_with_parser_and_invoker.invoke({"input": "Hey, can you turn the radio on?"})

Wow, that worked! Now let's turn it off:

In [ ]:
chain_with_parser_and_invoker.invoke({"input": "Hey, the radio volume is too high, can you turn it down please? Thanks."})

Too cool!!
The LLM is performing the Natural Language Understanding (NLU) task of converting commands in natural language into calls to specific actions (traditionally called *'Frames'* in NLP).

Let's see if the model can turn the volume up and down:

In [ ]:
chain_with_parser_and_invoker.invoke({"input": "Now I can hardly hear it. Please turn up the radio."})

In [ ]:
chain_with_parser_and_invoker.invoke({"input": "Hey, stop that radio. It's terrible!"})

Have a play around with the text to see whether the model is robust to different ways of requesting the same activity.

If we integrated the above functionality with a speech-to-text model, we would have a voice-interactive system for controlling the radio.

##Many more things to try!

There are lots of different directions for exploring LangChain functionality:
- Use Google New's queryable RSS feed (*'https://news.google.com/rss/search?hl=en-US&q='* + topic) to create a news-headline search tool, that is then used by an LLM (in a RAG model) to provide users with news updates on a particular topic.
- One very interesting idea is to create a ReAct (Reasoning Action) agent, which uses an updated version of LangChain to (see https://python.langchain.com/docs/how_to/chatbots_tools/) to easily integrate the results of tool invocations into the conversation.